In [49]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import BaseTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_ollama import ChatOllama
from duckduckgo_search import DDGS
from langchain_core.tools import tool
import os
import langchain

In [23]:
API_KEY = "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"
os.environ["TAVILY_API_KEY"] = API_KEY

In [38]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]


In [3]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [44]:
class Agent:
    def __init__(self, model, tools, checkpointer, system_msg=""):

        self.system_msg = system_msg

        self.graph = StateGraph(AgentState)
        self.graph.add_node("llm", self.call_model)
        self.graph.add_node("action", self.take_action)
        self.graph.add_conditional_edges(
            "llm",
            self.action_exists, 
            {
                True: "action",
                False: END,
            }
        )
        self.graph.add_edge("action", "llm")
        self.graph.set_entry_point("llm")
        self.graph = self.graph.compile(checkpointer=checkpointer)
        # what does checkpointer do?

        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
    
    def call_model(self, state: AgentState):
        messages = state["messages"]
        if self.system_msg:
            messages = [SystemMessage(content=self.system_msg)] + messages
        
        response = self.model.invoke(messages)
        return {"messages": [response]}
    
    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for tool_call in tool_calls:
            print(f"Executing tool call: {tool_call['name']}({tool_call['args']})")
            result = self.tools[tool_call['name']].invoke(tool_call['args'])
            results.append(ToolMessage(tool_call_id=tool_call['id'], name=tool_call['name'], content=str(result)))
        
        return {"messages": results}

    def action_exists(self, state: AgentState):
        last_message = state["messages"][-1]
        return len(last_message.tool_calls) > 0

In [40]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

In [50]:
model = ChatOllama(model="llama3.1:8b")
# tool = TavilySearchResults(max_results=5)

# DDGS().text("What is the capital of France?", max_results=1)

# @tool
# def search_tool(query: str, max_results=2):
#     """
#     Search the web for information
    
#     Args:
#         query: The query to search for
#         max_results: The maximum number of results to return
    
#     Returns:
#         A list of results
#     """
#     return DDGS().text(query, max_results=max_results)

search_tool = TavilySearchResults(max_results=5)
search_tool.invoke("What is the capital of France?")



[tool/start] [tool:tavily_search_results_json] Entering Tool run with input:
"What is the capital of France?"
[tool/end] [tool:tavily_search_results_json] [2.62s] Exiting Tool run with output:
"[{'url': 'https://home.adelphi.edu/~ca19535/page%204.html', 'content': 'Paris Paris, France Paris facts: Paris, the capital of France Paris is the capital of France, the largest country of Europe with 550 000 km2 (65 millions inhabitants). Paris is a world capital city of shopping and fashion, with Channel, Dior, Vuitton, Yves Saint Laurent among many other top french fashion brands. Paris facts: the capital of France in history Before Paris, the capital of France was Lyon (under the Romans). Paris first became the capital of France in 508 under King Clovis. After centuries with no unique capital of France, Paris retrieved its status of capital of France under King Philippe Auguste, who reigned between 1180 and 1223. Paris remained the capital of France until today, with one four year interrupti

[{'url': 'https://home.adelphi.edu/~ca19535/page%204.html',
  'content': 'Paris Paris, France Paris facts: Paris, the capital of France Paris is the capital of France, the largest country of Europe with 550 000 km2 (65 millions inhabitants). Paris is a world capital city of shopping and fashion, with Channel, Dior, Vuitton, Yves Saint Laurent among many other top french fashion brands. Paris facts: the capital of France in history Before Paris, the capital of France was Lyon (under the Romans). Paris first became the capital of France in 508 under King Clovis. After centuries with no unique capital of France, Paris retrieved its status of capital of France under King Philippe Auguste, who reigned between 1180 and 1223. Paris remained the capital of France until today, with one four year interruption.'},
 {'url': 'https://en.wikipedia.org/wiki/Paris',
  'content': 'Paris Paris Paris Paris (French pronunciation: [paʁi] ⓘ) is the capital and largest city of France. With 200,000 inhabitant

In [29]:
agent = Agent(model, [tool], checkpointer=memory)

In [52]:
messages = [HumanMessage(content="How is the weather in Delhi, India?")]
thread = {"configurable": {"thread_id": "1"}}

In [53]:
langchain.debug = True


In [54]:

with SqliteSaver.from_conn_string(":memory:") as memory:
    agent = Agent(model, [search_tool], checkpointer=memory)
    for event in agent.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v["messages"])


[chain/start] [chain:LangGraph] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:__start__] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:__start__ > chain:ChannelWrite<...>] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraph > chain:__start__ > chain:ChannelWrite<...>] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:LangGraph > chain:__start__ > chain:ChannelWrite<start:llm>] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraph > chain:__start__ > chain:ChannelWrite<start:llm>] s] Exiting Chain run with output:
[outputs]
[chain/end] [chain:LangGraph > chain:__start__] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:LangGraph > chain:llm] Entering Chain run with input:
[inputs]
[llm/start] [chain:LangGraph > chain:llm > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: How is the weather in Delhi, India?"
  ]
}
[llm/end] [chain:LangG

In [57]:
langchain.debug = False


with SqliteSaver.from_conn_string(":memory:") as memory:
    agent = Agent(model, [search_tool], checkpointer=memory)
    messages = [HumanMessage(content="And in mumbai?")]
    for event in agent.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v["messages"])

# TODO: need to use memory without context



[AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T10:59:42.330803Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1845785000, 'load_duration': 35503000, 'prompt_eval_count': 193, 'prompt_eval_duration': 1373000000, 'eval_count': 24, 'eval_duration': 435000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-68c706a2-bc1b-4232-95dc-f5bf67c40cc8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'mumbai current events'}, 'id': '22e220d7-490d-4519-b6c6-8588ff939c6f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 193, 'output_tokens': 24, 'total_tokens': 217})]
Executing tool call: tavily_search_results_json({'query': 'mumbai current events'})
[ToolMessage(content='[{\'url\': \'https://timesofindia.indiatimes.com/city/mumbai\', \'content\': \'Check out the latest news in Mumbai on The Times of India with a wide range of topics including Mumbai po